In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob
from datetime import datetime

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)


In [2]:
querys={
    
    
    'Produto':
    
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """
    
}

In [3]:
arquivos=glob('*.csv')

df=pd.read_csv(arquivos[-1],encoding='UTF-8')

df.columns

Index(['systemId', 'Code', 'Quantity', 'Amount', 'Sale Date',
       'Transaction ID'],
      dtype='object')

In [4]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [5]:
tabelas_df['Produto'].columns

Index(['SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Grupo de Produtos',
       'Grupo Indústria', 'Grupo de Meta', 'Grupo MCCAIN', 'Linha Fabrica',
       'Grupo de Promotores', 'Peso Liquido', 'Peso Bruto', 'Fator CX',
       'Unid. CMP', 'Unid. STK', 'Fator CMP', 'Fotos', 'EAN', 'DUN',
       'Peso Liquido Caixa', 'Peso Bruto Caixa', 'Fator Unid CMP'],
      dtype='object')

In [6]:
tabelas_df['Produto'].loc[tabelas_df['Produto']['Produto'].str.contains('BINT')]

,SKU,Cód. Fabricante,Produto,Status,Fabricante,Departamento,Seção,Categoria,Linha,Grupo de Produtos,Grupo Indústria,Grupo de Meta,Grupo MCCAIN,Linha Fabrica,Grupo de Promotores,Peso Liquido,Peso Bruto,Fator CX,Unid. CMP,Unid. STK,Fator CMP,Fotos,EAN,DUN,Peso Liquido Caixa,Peso Bruto Caixa,Fator Unid CMP
643,9337,20.1.01.17,"CONGELADO BATATA MCCAIN BINT 9MM (2,5KG)",ATIVO,MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA TRADICIONAL,BATATA GRADO B,MCCAIN,OUTRAS MARCAS,MCCAIN FOOD,MCCAIN FOOD,BATATA GRADO B,DE MARCHI,2.5,2.5,6.0,KG,UN,2.5,\\192.168.0.252\Progins\MOINHO\fotos_dos_produ...,7797906001141,17797906001148,15.0,15.0,0.4
659,9362,1000002255,CONG. BATATA MCCAIN BINT 9MM (5X2KG),ATIVO,MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA VAREJO,BATATA PALITO,MCCAIN,OUTRAS MARCAS,MCCAIN VAREJO,MCCAIN VAREJO,BATATA PALITO,DE MARCHI,2.0,2.0,5.0,KG,UN,2.0,\\192.168.0.252\Progins\MOINHO\fotos_dos_produ...,7797906001738,17797906001735,10.0,10.0,0.5
700,9403,1000009718,CONG. BATATA MCCAIN BINT 9MM (8X2KG),ATIVO,MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA VAREJO,BATATA PALITO,MCCAIN,OUTRAS MARCAS,MCCAIN VAREJO,MCCAIN VAREJO,BATATA PALITO,DE MARCHI,2.0,2.0,8.0,CX,UN,8.0,\\192.168.0.252\Progins\MOINHO\fotos_dos_produ...,7896105800682,17896105800689,16.0,16.0,8.0
829,9532,,CONG. BATATA MCCAIN BINT IMPORT 9MM (2KG),INATIVO,MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA VAREJO,BATATA PALITO,MCCAIN,OUTRAS MARCAS,MCCAIN VAREJO,MCCAIN VAREJO,BATATA PALITO,DE MARCHI,2.0,2.0,8.0,KG,UN,2.0,\\192.168.0.252\Progins\MOINHO\fotos_dos_produ...,7797906001738,27797906001732,16.0,16.0,0.5


In [7]:
df=df.merge(tabelas_df['Produto'],left_on='Code',right_on='SKU',how='inner')

In [8]:
df['Sale Date']=pd.to_datetime(df['Sale Date'])

In [9]:
df['ID Mês']=df['Sale Date'].dt.month

In [10]:
produtos_df=pd.DataFrame()

produtos_df=df[['SKU','Produto','Fabricante','Grupo MCCAIN','Quantity','Amount']].groupby(['SKU','Produto','Fabricante','Grupo MCCAIN'],as_index=False).sum()

In [11]:
def Mes(id):
    
    meses={
        
        1:'JAN',
        2:'FEV',
        3:'MAR',
        4:'ABR',
        5:'MAI',
        6:'JUN',
        7:'JUL',
        8:'AGO',
        9:'SET',
        10:'OUT',
        11:'NOV',
        12:'DEZ',
    }
    
    return meses[id]
    
    pass

In [12]:
temp_df=pd.DataFrame()

temp_df=df[['SKU','ID Mês','Quantity','Amount']].groupby(['SKU','ID Mês'],as_index=False).sum()


In [13]:
temp_df=temp_df.pivot(index='SKU',columns='ID Mês',values='Amount').reset_index()

In [14]:
for i in temp_df.columns[1:].to_list():
    
    mes=Mes(i)
    
    temp_df.rename(columns={i:mes},inplace=True)
    
    pass

temp_df

ID Mês,SKU,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT
0,594,172233.30,152021.04,162999.98,142997.68,183112.85,258114.55,199717.81,233172.45,229285.59,112912.80
1,614,2847.76,9491.28,9330.28,7671.99,8447.85,6711.41,NaN,NaN,NaN,NaN
2,716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1448.12,NaN,NaN
3,787,97792.18,24100.86,37990.87,132532.12,572.38,10.00,NaN,96099.54,34743.90,9135.00
4,815,1112.25,2018.40,12953.72,15805.72,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
56,9585,NaN,NaN,NaN,2898.88,11494.34,NaN,NaN,NaN,NaN,NaN
57,9586,NaN,NaN,NaN,NaN,963.10,1505.24,1870.85,4312.32,963.60,NaN
58,9589,NaN,NaN,NaN,NaN,NaN,8273.30,7166.09,1502.59,NaN,NaN
59,9592,NaN,NaN,NaN,NaN,NaN,NaN,3257.41,5245.97,15677.44,5844.27


In [17]:
produtos_df=produtos_df.merge(temp_df,on='SKU',how='inner')

In [19]:
produtos_df.loc[produtos_df['Produto'].str.contains('BINT')]

,SKU,Produto,Fabricante,Grupo MCCAIN,Quantity,Amount,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT
33,9337,"CONGELADO BATATA MCCAIN BINT 9MM (2,5KG)",MCCAIN,MCCAIN FOOD,29090,776990.57,3600.84,44888.55,94949.72,80417.36,219404.8,22682.64,152694.38,114019.94,67.84,44264.5
35,9362,CONG. BATATA MCCAIN BINT 9MM (5X2KG),MCCAIN,MCCAIN VAREJO,1320,17828.87,23.00,17757.90,0.00,47.97,NaN,NaN,NaN,NaN,NaN,NaN
43,9532,CONG. BATATA MCCAIN BINT IMPORT 9MM (2KG),MCCAIN,MCCAIN VAREJO,541,8291.79,7488.92,133.36,NaN,NaN,NaN,NaN,NaN,669.51,NaN,NaN
